### Imports

In [1]:
import urllib.request
import re
from importlib.metadata import version

import torch
from torch.utils.data import Dataset, DataLoader

import tiktoken
print('tiktoken version: ', version('tiktoken'))

tiktoken version:  0.9.0


### Download Data

The file is available at https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/the-verdict.txt

In [2]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
  raw_text = f.read()
print('Total number of characters: ', len(raw_text))
print(raw_text[:99])

Total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Preprocess and create vocabulary

In [3]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])
all_words = set(preprocessed)
vocab_size = len(all_words)
print(vocab_size)

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
1130


In [4]:
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
  print(item)
  if i >= 50:
    break

('strain', 0)
('Be', 1)
('surprise', 2)
('widow', 3)
('rich', 4)
('eighteenth-century', 5)
('just', 6)
('at', 7)
('point', 8)
('Don', 9)
('any', 10)
('chest', 11)
('wits', 12)
('thither', 13)
('unusual', 14)
('effects', 15)
('mourn', 16)
('behind', 17)
('adulation', 18)
('rest', 19)
('unexpected', 20)
('equally', 21)
('past', 22)
('outline', 23)
('fluently', 24)
('quality', 25)
('almost', 26)
('before', 27)
('flung', 28)
('unaccountable', 29)
('showed', 30)
('little', 31)
('heart', 32)
('loathing', 33)
('shrug', 34)
('bespoke', 35)
('find', 36)
('cleverer', 37)
('And', 38)
('prove', 39)
('light', 40)
('faces', 41)
('apparently', 42)
('their', 43)
('me', 44)
('place', 45)
('extracting', 46)
('deadening', 47)
('manage', 48)
('twirling', 49)
('absorbed', 50)


### Extend vocabulary by special tokens

In [5]:
all_tokens = sorted(list(preprocessed))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

1132
('younger', 4685)
('your', 4688)
('yourself', 4689)
('<|endoftext|>', 4690)
('<|unk|>', 4691)


### Create and Test a simple Tokenizer

In [6]:
class SimpleTokenizer:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s, i in vocab.items()}

  def encode(self, raw_text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[id] for id in ids])
    text = re.sub(r'\s+([,.:;?"()\'])', r'\1', text)
    return text

In [7]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace of someunknownPlace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace of someunknownPlace.


In [8]:
tokenizer = SimpleTokenizer(vocab)
token_ids = tokenizer.encode(text)
print(token_ids)
decoded_text = tokenizer.decode(token_ids)
print(decoded_text)

[4691, 489, 2009, 4684, 2906, 3889, 865, 4690, 1102, 4131, 3836, 3903, 3265, 4131, 4691, 3265, 4691, 798]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|> of <|unk|>.


### Use GPT2 Tokenizer

In [9]:
tokenizer = tiktoken.get_encoding('gpt2')
tik_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(tik_ids)
strings = tokenizer.decode(tik_ids)
print(strings)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 20562, 286, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace of someunknownPlace.


In [10]:
strange_word = 'Akwirw ier'
ids = tokenizer.encode(strange_word)
print(ids)
word = tokenizer.decode(ids)
print(word)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


### Create Tokenized Dataset and Dataloaders using GPT2 Tokenizer

In [11]:
class GPTDatasetV1(Dataset):
  def __init__(self, text, tokenizer, max_length, stride):
    self.input_ids, self.target_ids = [], []
    self.max_length = max_length
    self.stride = stride

    tokens = tokenizer.encode(text)

    for i in range(0, len(tokens)-self.max_length, self.stride):
      self.input_ids.append(torch.tensor(tokens[i:i+self.max_length]))
      self.target_ids.append(torch.tensor(tokens[i+1:i+self.max_length+1]))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [12]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
  tokenizer = tiktoken.get_encoding('gpt2')
  dataset = GPTDatasetV1(txt, tokenizer, max_length=max_length, stride=stride)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
  return dataloader

def get_batch(dataloader):
  data_iter = iter(dataloader)
  print('First batch: ', next(data_iter))
  print('Second batch: ', next(data_iter))

In [13]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
  raw_text = f.read()

In [14]:
dataloader_1 = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
get_batch(dataloader_1)

dataloader_2 = create_dataloader_v1(raw_text, batch_size=1, max_length=2, stride=2, shuffle=False)
get_batch(dataloader_2)

dataloader_3 = create_dataloader_v1(raw_text, batch_size=1, max_length=8, stride=2, shuffle=False)
get_batch(dataloader_3)

dataloader_4 = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
get_batch(dataloader_4)

# avoid overlap ---> avoid overfitting

First batch:  [tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
Second batch:  [tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]
First batch:  [tensor([[ 40, 367]]), tensor([[ 367, 2885]])]
Second batch:  [tensor([[2885, 1464]]), tensor([[1464, 1807]])]
First batch:  [tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]
Second batch:  [tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]
First batch:  [tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        